In [2]:
!pip install efficientnet #>> /dev/null/

     |████████████████████████████████| 50 kB 2.3 MB/s eta 0:00:011


In [3]:
import tensorflow as tf
import keras 
from keras.applications import DenseNet201
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import models, layers, optimizers
import tensorflow.keras.backend as K
import os
import efficientnet.tfkeras as efn
from keras.utils import np_utils

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.list_physical_devices('GPU'); print(gpus)
if len(gpus)==1: strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
else: strategy = tf.distribute.MirroredStrategy()

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
print('Mixed precision enabled')

Mixed precision enabled


In [6]:
X = np.load("../input/pcb-image-dataset1/xtrain.npy")
y = np.load("../input/pcb-image-dataset1/ytrain.npy")

In [7]:
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

datagen.fit(X)

In [8]:
y = np_utils.to_categorical(y)

In [9]:
 conv_base = efn.EfficientNetB4(weights='imagenet',include_top=False, input_shape=(300,300,3))

71892992/71892840 [==============================] - 1s 0us/step


In [ ]:
conv_base(X[0].reshape(1,300,300,3)).shape

In [ ]:
del X
del y

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=8)

In [ ]:
# f = 1
# accuracies = []
# for train_index, test_index in skf.split(samples, labels):
#     print('Fold {}'.format(f))
#     X_train = samples[train_index]
#     X_test = samples[test_index]
#     y_train = labels[train_index]
#     y_test = labels[test_index]
#     y_train = np_utils.to_categorical(y_train)
#     y_test = np_utils.to_categorical(y_test)

#     K.clear_session()
#     with strategy.scope():
#         model = models.Sequential()
#         model.add(layers.Dense(256, activation = 'relu', input_dim = 10*10*1792))
#         model.add(layers.Dropout(0.5))
#         model.add(layers.Dense(2, activation = 'softmax'))
#         model.compile(optimizer = optimizers.Adam(lr = 0.00001),loss = 'categorical_crossentropy',metrics = ['categorical_accuracy'])
#         my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]
#         history = model.fit(X_train, y_train, epochs = 100, batch_size = 8, validation_data = (X_test, y_test), shuffle = True, callbacks = my_callbacks)
            
    
#         test_preds = model.predict(X_test, batch_size = 20)
#         test_predclass = np.argmax(test_preds, axis=1)
#         truth = np.argmax(y_test,axis=1)
        
#         accuracies.append([confusion_matrix(truth,test_predclass)[0][0]/(truth == 0).sum(), confusion_matrix(truth,test_predclass)[1][1]/(y_test == 1).sum()])
#         f+=1


# zero_acc = 0
# one_acc = 0

# for i in accuracies:
#     zero_acc+= i[0]
#     one_acc+=i[1]

# zero_acc/=8
# one_acc/=8

# print("Accuracies: ",accuracies)

In [ ]:
for i in accuracies:
    print(i)

In [ ]:
zero_acc, one_acc

Test Set 

In [10]:
Xtest = np.load("../input/pcb-image-dataset1/xtest.npy")
ytest = np.load("../input/pcb-image-dataset1/ytest.npy")

In [11]:
ytest = np_utils.to_categorical(ytest)

In [12]:
datagentest = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

datagentest.fit(Xtest)

In [14]:
for i in conv_base.layers:
    i.trainable = False

t = [i.trainable for i in conv_base.layers]


In [16]:
# CUSTOM LEARNING SCHEUDLE
LR_START = 1e-5
LR_MAX = 1e-3
LR_RAMPUP_EPOCHS = 5
LR_SUSTAIN_EPOCHS = 0
LR_STEP_DECAY = 0.75

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = LR_MAX * LR_STEP_DECAY**((epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS)//10)
    return lr
    
lr2 = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

In [28]:
K.clear_session()

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b4 (Model)      (None, 10, 10, 1792)      17673816  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1792)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 3586      
Total params: 17,677,402
Trainable params: 3,586
Non-trainable params: 17,673,816
_________________________________________________________________


In [29]:
# with strategy.scope():
model = models.Sequential()
model.add(conv_base)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(2, activation = 'softmax'))
model.compile(optimizer = optimizers.Adam(lr = 0.00001),loss = 'categorical_crossentropy',metrics = ['categorical_accuracy'])
my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]
    
history = model.fit(X, y, epochs = 100, batch_size = 8, validation_data = (Xtest, ytest), shuffle = True, callbacks = my_callbacks)

Epoch 1/100
59/59 [==============================] - 10s 169ms/step - loss: 5.2062 - categorical_accuracy: 0.5297 - val_loss: 1.1791 - val_categorical_accuracy: 0.4384
Epoch 2/100
59/59 [==============================] - 7s 120ms/step - loss: 5.8338 - categorical_accuracy: 0.4640 - val_loss: 1.0982 - val_categorical_accuracy: 0.4674
Epoch 3/100
59/59 [==============================] - 7s 119ms/step - loss: 3.5028 - categorical_accuracy: 0.5254 - val_loss: 1.0239 - val_categorical_accuracy: 0.4891
Epoch 4/100
59/59 [==============================] - 7s 118ms/step - loss: 4.1365 - categorical_accuracy: 0.4958 - val_loss: 0.9894 - val_categorical_accuracy: 0.5054
Epoch 5/100
59/59 [==============================] - 7s 118ms/step - loss: 4.4508 - categorical_accuracy: 0.4936 - val_loss: 0.9249 - val_categorical_accuracy: 0.5326
Epoch 6/100
59/59 [==============================] - 7s 118ms/step - loss: 3.8860 - categorical_accuracy: 0.5085 - val_loss: 0.9124 - val_categorical_accuracy: 0.53

In [31]:
for i in conv_base.layers:
    print(i.name)

input_1
stem_conv
stem_bn
stem_activation
block1a_dwconv
block1a_bn
block1a_activation
block1a_se_squeeze
block1a_se_reshape
block1a_se_reduce
block1a_se_expand
block1a_se_excite
block1a_project_conv
block1a_project_bn
block1b_dwconv
block1b_bn
block1b_activation
block1b_se_squeeze
block1b_se_reshape
block1b_se_reduce
block1b_se_expand
block1b_se_excite
block1b_project_conv
block1b_project_bn
block1b_drop
block1b_add
block2a_expand_conv
block2a_expand_bn
block2a_expand_activation
block2a_dwconv
block2a_bn
block2a_activation
block2a_se_squeeze
block2a_se_reshape
block2a_se_reduce
block2a_se_expand
block2a_se_excite
block2a_project_conv
block2a_project_bn
block2b_expand_conv
block2b_expand_bn
block2b_expand_activation
block2b_dwconv
block2b_bn
block2b_activation
block2b_se_squeeze
block2b_se_reshape
block2b_se_reduce
block2b_se_expand
block2b_se_excite
block2b_project_conv
block2b_project_bn
block2b_drop
block2b_add
block2c_expand_conv
block2c_expand_bn
block2c_expand_activation
block2c_